Импортируем необходимые для работы библиотеки

In [1]:
!pip install torchmetrics

     |████████████████████████████████| 332 kB 5.4 MB/s 


In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import shuffle
from collections import Counter
import gensim.downloader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import torchmetrics
from torchmetrics import F1
from torchmetrics.functional import f1, recall
import statistics

nltk.download('punkt')
le = preprocessing.LabelEncoder()
torch.manual_seed(42)
np.random.seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Fake.csv')

In [4]:
df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


В качестве препроцессинга авторы только приводят текст к нижнему регистру, падят тексты до максимальной длины, и векторизуют готовой моделью ворт2века. Поэтому мы сделаем то же самое, но при паддинге посмотрим на самую длинную последовательность, потому что работаем на других данных

In [5]:
df['preprocessed_text'] = df['text'].apply(lambda x: word_tokenize(x.lower()))

In [6]:
df

,title,text,subject,date,preprocessed_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[donald, trump, just, couldn, t, wish, all, am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[house, intelligence, committee, chairman, dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[on, friday, ,, it, was, revealed, that, forme..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[on, christmas, day, ,, donald, trump, announc..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[pope, francis, used, his, annual, christmas, ..."
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016","[21st, century, wire, says, as, 21wire, report..."
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016","[21st, century, wire, says, it, s, a, familiar..."
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016","[patrick, henningsen, 21st, century, wireremem..."
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016","[21st, century, wire, says, al, jazeera, ameri..."


In [7]:
df['target'] = df['subject'].astype('category').cat.codes

In [8]:
le.fit(list(df.subject))

LabelEncoder()

In [9]:
le.classes_

array(['Government News', 'Middle-east', 'News', 'US_News', 'left-news',
       'politics'], dtype='<U15')

In [10]:
df["target"] = list(le.transform(list(df.subject)))

In [11]:
df["preprocessed_2"] = df['preprocessed_text'].apply(lambda x: x[:500])

In [12]:
df

,title,text,subject,date,preprocessed_text,target,preprocessed_2
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[donald, trump, just, couldn, t, wish, all, am...",2,"[donald, trump, just, couldn, t, wish, all, am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[house, intelligence, committee, chairman, dev...",2,"[house, intelligence, committee, chairman, dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[on, friday, ,, it, was, revealed, that, forme...",2,"[on, friday, ,, it, was, revealed, that, forme..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[on, christmas, day, ,, donald, trump, announc...",2,"[on, christmas, day, ,, donald, trump, announc..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[pope, francis, used, his, annual, christmas, ...",2,"[pope, francis, used, his, annual, christmas, ..."
...,...,...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016","[21st, century, wire, says, as, 21wire, report...",1,"[21st, century, wire, says, as, 21wire, report..."
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016","[21st, century, wire, says, it, s, a, familiar...",1,"[21st, century, wire, says, it, s, a, familiar..."
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016","[patrick, henningsen, 21st, century, wireremem...",1,"[patrick, henningsen, 21st, century, wireremem..."
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016","[21st, century, wire, says, al, jazeera, ameri...",1,"[21st, century, wire, says, al, jazeera, ameri..."


In [13]:
len(list(df.preprocessed_2)[0])

500

In [14]:
news_data = shuffle(df[['preprocessed_2','target']])

In [15]:
train_texts, val_texts = train_test_split(news_data, test_size=0.2)

In [16]:
train_texts[:10]

,preprocessed_2,target
18375,"[when, cnn, ,, who, is, owned, by, multi-billi...",4
7847,"[a, male, idaho, republican, with, five, daugh...",2
16802,"[you, seriously, can, t, make, this, up, ., th...",0
10622,"[virginia, governor, terry, mcauliffe, decided...",5
8956,"[president, obama, could, not, hold, back, the...",2
11170,"[the, syrian, refugee, who, shocked, cnn, was,...",5
22810,"[21st, century, wire, says, the, culture, wars...",1
10262,"[do, you, just, want, to, wring, this, guy, s,...",5
20998,"[all, of, a, sudden, #, blackdeathsmatter, cou...",4
19712,"[this, is, hands, down, the, best, commentary,...",4


In [17]:
vocab = Counter()

for text in news_data['preprocessed_2']:
    vocab.update(text)
print('всего уникальных токенов:', len(vocab))

всего уникальных токенов: 151903


In [18]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

уникальных токенов, втретившихся больше 2 раз: 49898


In [19]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [20]:
id2word = {i:word for word, i in word2id.items()}

In [132]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [133]:
max(len(elem) for elem in list(df.preprocessed_text))


9938

Чтобы колаб меня не забанил так такие длинющие паддинги, выберем длину самостоятельно

In [134]:
statistics.mean(len(elem) for elem in list(df.preprocessed_text))

469.4998083556918

Возьмем длину 500

In [135]:
class NewsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['preprocessed_2'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['target'].values
        self.device = DEVICE

    def __len__(self):
        return self.length

    def __getitem__(self, index): 
        words = self.dataset[index]
        ids = torch.LongTensor([self.word2id[word] for word in words if word in self.word2id])
        y = [self.target[index]]
        return ids, y

    def collate_fn(self, batch): 
      ids, y = list(zip(*batch))
      ids_500 = [(l[:500]) for l in ids]
      # print(ids[0])
      # padded_ids = nn.ConstantPad1d(500, ids).to(self.device)
      # print(type(padded_ids))
      padded_ids = pad_sequence(ids_500, batch_first=True).to(self.device)
      # print(type(padded_ids))
      y = torch.Tensor(y).to(self.device)
      return padded_ids, y

In [136]:
train_dataset = NewsDataset(train_texts, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=64, drop_last=True)

In [137]:
train_texts

,preprocessed_2,target
18375,"[when, cnn, ,, who, is, owned, by, multi-billi...",4
7847,"[a, male, idaho, republican, with, five, daugh...",2
16802,"[you, seriously, can, t, make, this, up, ., th...",0
10622,"[virginia, governor, terry, mcauliffe, decided...",5
8956,"[president, obama, could, not, hold, back, the...",2
...,...,...
1015,"[ever, since, same-sex, marriage, became, the,...",2
23276,"[tune, in, to, the, alternate, current, radio,...",1
17413,"[as, it, turns, out, ,, 67, year, old, hillary...",0
19876,"[the, office, of, immigration, statistics, rep...",4


In [138]:
batch = next(iter(train_iterator))

In [139]:
batch[0]

tensor([[ 3611, 37801, 29216,  ...,     0,     0,     0],
        [  746, 26162, 13887,  ...,     0,     0,     0],
        [18855,  5988, 13780,  ...,     0,     0,     0],
        ...,
        [18855,  5988, 13780,  ...,     0,     0,     0],
        [ 7781, 30733,   746,  ...,     0,     0,     0],
        [31106, 36760, 44472,  ...,     0,     0,     0]], device='cuda:0')

In [140]:

[id2word[int(i)] for i in batch[0][0]]

['nobody',
 'with',
 'any',
 'brains',
 'has',
 'ever',
 'accused',
 'rush',
 'limbaugh',
 'of',
 'understanding',
 'pretty',
 'much',
 'anything',
 'at',
 'all',
 ',',
 'but',
 'his',
 'latest',
 'bout',
 'with',
 'just',
 'went',
 'way',
 'over',
 'the',
 'line',
 ',',
 'even',
 'for',
 'him',
 '.',
 'while',
 'whining',
 'up',
 'a',
 'storm',
 'about',
 'decaying',
 'sexual',
 'morality',
 'in',
 'this',
 'country',
 ',',
 'he',
 'tried',
 'to',
 'paint',
 'the',
 'left',
 'as',
 'hypocrites',
 'because',
 'we',
 'tolerate',
 'anything',
 'sexual',
 'as',
 'long',
 'as',
 'there',
 's',
 'consent',
 '.',
 'when',
 'there',
 's',
 'no',
 'consent',
 ',',
 'we',
 'send',
 'out',
 'the',
 'rape',
 'police',
 ',',
 'and',
 'apparently',
 ',',
 'this',
 'is',
 'a',
 'you',
 'ever',
 'heard',
 'anything',
 'so',
 'completely',
 'asinine',
 'in',
 'your',
 'life',
 '?',
 'what',
 'people',
 'do',
 'in',
 'their',
 'bedrooms',
 'is',
 'their',
 'business',
 ',',
 'but',
 'all',
 'bets',
 'a

In [141]:
batch[1]

tensor([[2.],
        [2.],
        [3.],
        [2.],
        [4.],
        [2.],
        [3.],
        [2.],
        [2.],
        [5.],
        [0.],
        [5.],
        [5.],
        [4.],
        [2.],
        [5.],
        [5.],
        [2.],
        [2.],
        [2.],
        [2.],
        [4.],
        [2.],
        [4.],
        [5.],
        [4.],
        [2.],
        [2.],
        [4.],
        [5.],
        [5.],
        [2.],
        [0.],
        [5.],
        [2.],
        [2.],
        [5.],
        [2.],
        [4.],
        [2.],
        [5.],
        [0.],
        [5.],
        [2.],
        [2.],
        [2.],
        [5.],
        [2.],
        [0.],
        [5.],
        [2.],
        [4.],
        [2.],
        [2.],
        [2.],
        [2.],
        [5.],
        [3.],
        [2.],
        [4.],
        [2.],
        [1.],
        [4.],
        [5.]], device='cuda:0')

In [142]:
val_dataset = NewsDataset(val_texts, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=64)

In [143]:
test_batch = next(iter(val_iterator))
test_batch[0].shape

torch.Size([64, 500])

In [32]:
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [144]:
weights = np.zeros((len(word2id), 300))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = w2v_vectors.wv[word]    
    except KeyError:
      count += 1
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(-0.25,0.25,300)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [180]:
class C_LSTM(nn.Module):
    
    def __init__(self, filter_lengths, dropout_layer, vocab_size, embedding_dim, weights, n_filters,
                 hidden_dim, output_dim, dropout):
        
        super().__init__()

        self.emb_layer = nn.Embedding(vocab_size, embedding_dim)
        self.emb_layer.from_pretrained(torch.tensor(weights), freeze=True)
        self.filter_lengths = filter_lengths
        self.dropout_layer = dropout_layer

        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=n_filters,
                                  kernel_size=2)
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=n_filters,
                                    kernel_size=3)
        
        self.fourgrams = nn.Conv1d(in_channels=embedding_dim, out_channels=n_filters,
                                   kernel_size=4)
      
        self.lstm = nn.LSTM(input_size=n_filters, hidden_size=hidden_dim,
                            batch_first=True)

        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        # self.out = nn.Softmax(dim=1)
        self.linear = nn.Linear(in_features=hidden_dim, out_features=output_dim)

    def forward(self, x):
        x_emb = self.emb_layer(x)
        if self.dropout_layer == 0:
          x_emb = self.dropout(x_emb)

        x_emb = x_emb.transpose(1,2)

        max_len_feat = (500 - max(self.filter_lengths)) + 1
        conv = []
        for f_len in self.filter_lengths:
          if f_len == 2:
            f_map = self.bigrams(x_emb)
            f_map = self.relu(f_map)
            f_map = f_map[:, :, :max_len_feat]
            conv.append(f_map)

          elif f_len == 3:
            f_map = self.trigrams(x_emb)
            f_map = self.relu(f_map)
            f_map = f_map[:, :, :max_len_feat]
            conv.append(f_map)
          
          else:
            f_map = self.fourgrams(x_emb)
            f_map = self.relu(f_map)
            f_map = f_map[:, :, :max_len_feat]
            conv.append(f_map)


        if len(self.filter_lengths) != 1:
            concat_ngrams = torch.cat(conv, -1)
        else:
            concat_ngrams = f_map

        concat_ngrams = concat_ngrams.transpose(1, 2)
        output, (hidden, cell) = self.lstm(concat_ngrams)
        if self.dropout_layer == 1:
          output = self.dropout(output)
        
        logits = self.linear(torch.squeeze(hidden, 0))  


        return logits

In [181]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0

    model.train()

    for i, (texts, ys) in enumerate(iterator): 
        optimizer.zero_grad()
        preds = model(texts)  
        ys = ys.to(torch.int64)
        loss = criterion(preds, ys.squeeze())  
        loss.backward() 
        optimizer.step() 
        epoch_loss += loss.item()
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator)

In [182]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    metric = torchmetrics.Accuracy()
    metric.to(DEVICE)
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts)
            # print(preds)
            # print(ys)
            ys = ys.to(torch.int64)
            loss = criterion(preds, ys.squeeze())
            epoch_loss += loss.item()
            batch_metric = metric(preds, ys.squeeze())
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
              print(f'Val loss: {epoch_loss/i}, Val accuracy: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator)

In [183]:
def experiment(filter_lengths, dropout_layer):
    
    model = C_LSTM(filter_lengths=filter_lengths, dropout_layer=dropout_layer,
                   vocab_size=len(word2id), embedding_dim=100, weights=weights,
               n_filters=150, hidden_dim=150, output_dim=6, dropout=0.5)

    model = model.to(DEVICE)

    optimizer = optim.Adam(model.parameters())

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)

    losses = []
    losses_eval = []
    accs = []
    accs_eval = []

    print("Пармеметры:")
    print(f'\n#######\n\n длины фильтров: {filter_lengths}')
    print(f'\nСлой, на котором применяется дропаут: {dropout_layer}')
    for i in range(5):
        print(f'\nstarting Epoch {i}')
        print('Training...')
        epoch_loss = train(model, train_iterator, optimizer, criterion)
        losses.append(epoch_loss)
        print('\nEvaluating on train...')
        acc_on_train,_ = evaluate(model, train_iterator, criterion)
        accs.append(acc_on_train)
        print('\nEvaluating on test...')
        acc_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
        losses_eval.append(epoch_loss_on_test)
        accs_eval.append(acc_on_test)


In [185]:
# 0 - делаем дропаут после эмбеддинг слоя
# 1 - делаем дропаут перед линейным слоем
params = [[[2], 0], [[2], 1], [[3], 0], [[3], 1], [[4], 0], [[4], 1],
          [[2, 3], 0], [[2, 3], 1], [[2, 4], 0], [[2, 4], 1],
          [[3, 4], 0], [[3, 4], 1], [[2, 3, 4], 0], [[2, 3, 4], 0]] 


for param in params:
    experiment(filter_lengths=param[0], dropout_layer=param[1])

Пармеметры:

#######

 длины фильтров: [2]

Слой, на котором применяется дропаут: 0

starting Epoch 0
Training...
Train loss: 1.5069096673998916
Train loss: 1.4788073290949282
Train loss: 1.4660652503801908
Train loss: 1.457874445688157
Train loss: 1.454728629762326

Evaluating on train...
Val loss: 1.4666895029837625, Val accuracy: 0.38843202590942383
Val loss: 1.4386199536530868, Val accuracy: 0.39211955666542053
Val loss: 1.4264575780471624, Val accuracy: 0.3978504240512848
Val loss: 1.4231054199722422, Val accuracy: 0.39948591589927673
Val loss: 1.4256289067152874, Val accuracy: 0.3982482850551605

Evaluating on test...
Val loss: 1.5522707242232103, Val accuracy: 0.4086538553237915
Val loss: 1.4855808637760304, Val accuracy: 0.394097238779068
Val loss: 1.4542613058555416, Val accuracy: 0.3982469439506531
Val loss: 1.4463361349972812, Val accuracy: 0.3911931812763214
Val loss: 1.4398125237312869, Val accuracy: 0.38813406229019165

starting Epoch 1
Training...
Train loss: 1.416104433

### Выводы
Так как эксперимент очень большой, я решила запустить всего по 5 эпох, чтобы колаб меня не забанил 

Исходя из результатов эксперимента:

1) лучше всего справилась модель с одним Conv1d слоем, где длина фильтра равна четырем, и с дропаутом перед линейным слоем;

2) в среднем лучше справлялись модели с двумя Conv1d слоями;

3) модели с дропаутом перед линейным слоем работают лучше, чем модели с дропаутом после эмбеддинг-слоя